In [ ]:
!pip install biopython

In [1]:
!pip install PyVCF

  Created wheel for PyVCF: filename=PyVCF-0.6.8-cp37-cp37m-linux_x86_64.whl size=124145 sha256=23e4038d182aac47cff917930eff5232d5696c604342b1feb20060dd03f138e7
  Stored in directory: /root/.cache/pip/wheels/8a/60/8d/fe98f8401a0bb73b44afbbc454d1d7fe11a0a2081f5b899597
Successfully built PyVCF


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import vcf
from IPython.display import display
from ipywidgets import FloatProgress
%matplotlib inline
plt.style.use('ggplot')

In [3]:
brca1_variants = pd.read_csv(os.path.join('/content/drive/MyDrive/Quantitative_Biological_Research_with_Python/VCF_Multivar_Analysis/brca1_variants.csv'))
display(brca1_variants)

,HGVS Variant,Pathogenicity,Pathogenic,Start,End,Reference Seq,Variant Seq,Reference Seq Complement,Variant Seq Complement
0,c.8T>G,5,True,41276105,41276105,A,C,T,G
1,c.32_33insC,5,True,41276080,41276081,TA,TGA,TA,TCA
2,c.34C>T,5,True,41276079,41276079,G,A,C,T
3,c.38_39delATinsGGG,5,True,41276074,41276075,AT,CCC,AT,GGG
4,c.53T>C,4,True,41276060,41276060,A,G,T,C
...,...,...,...,...,...,...,...,...,...
1163,c.5553dupC,5,True,41197733,41197733,G,GG,C,CC
1164,c.5558dupA,5,True,41197728,41197728,T,TT,A,AA
1165,c.5559C>G,5,True,41197727,41197727,G,C,C,G
1166,c.5572A>C,1,False,41197714,41197714,T,G,A,C


In [4]:
# We hold a cache of all pathogenic variants in a set (for fast lookup). Each variant is represented as a tuple of:
# (location (0-based index; int), ref seq (str), alt seq (str)).

pathogenic_variants = set(brca1_variants[brca1_variants['Pathogenic']].apply(lambda record: \
        (record['Start'], record['Reference Seq'], record['Variant Seq']), axis = 1))

print(len(pathogenic_variants))
print(list(pathogenic_variants)[:5])

1085
[(41244887, 'GC', 'GTC'), (41226449, 'GAGA', nan), (41243478, 'CTTG', nan), (41244413, 'TTAATATTGC', nan), (41243854, 'A', 'AA')]


In [5]:
VCF_FILE_NAME = '/content/drive/MyDrive/Quantitative_Biological_Research_with_Python/VCF_Multivar_Analysis/ALL.chr17.integrated_phase1_v3.20101123.snps_indels_svs.genotypes.41100000-41300000.vcf.gz'

for variant in vcf.Reader(filename = os.path.join(VCF_FILE_NAME), compressed = True):
    for allele_index, alt in enumerate(variant.ALT):
        
        # variant_key will have the same representation as the elements in the lookup set of pathogenic variants.
        # Note that we convert the position from 1-based to 0-based index, and that an empty sequence is represented as np.nan.
        variant_key = (variant.POS - 1, str(variant.REF), np.nan if len(alt) == 0 else str(alt))
        
        if variant_key in pathogenic_variants:
            for call in variant.samples:
                # call.gt_alleles gives the genotypes of the current sample as numbers. 0 represents the ref allele, 1 represents
                # the first alternative allele, and so on. On the other hand, allele_index starts counting the first alternative
                # allele as 0. Therefore, we check whether (allele_index + 1) is in the list of allele numbers of the genotype
                # of the current sample. 
                if (allele_index + 1) in map(int, call.gt_alleles):
                    print('Sample %s has the variant %s --> %s at position %d (%s)' % \
                            (call.sample, variant.REF, alt, variant.POS, variant.ID))

Sample HG00403 has the variant T --> A at position 41201105 (rs80358092)
Sample HG00537 has the variant T --> A at position 41201105 (rs80358092)
Sample HG00626 has the variant T --> A at position 41201105 (rs80358092)
Sample NA18528 has the variant T --> A at position 41201105 (rs80358092)
Sample NA18570 has the variant T --> A at position 41201105 (rs80358092)
Sample NA18613 has the variant T --> A at position 41201105 (rs80358092)
Sample NA18641 has the variant T --> A at position 41201105 (rs80358092)
Sample NA19780 has the variant A --> C at position 41243840 (rs28897687)
Sample NA19380 has the variant C --> G at position 41243948 (rs56214134)
Sample NA19382 has the variant C --> G at position 41243948 (rs56214134)
